In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!pip install easyocr
!pip install SpeechRecognition
!pip install -U scikit-learn
!pip install twitter
!pip install flask-ngrok
!pip install cvlib

     |████████████████████████████████| 63.6 MB 55 kB/s 
     |████████████████████████████████| 47.6 MB 1.4 MB/s 
     |████████████████████████████████| 32.8 MB 54.0 MB/s 
     |████████████████████████████████| 50 kB 2.5 MB/s 
     |████████████████████████████████| 10.0 MB 4.2 MB/s 
  Created wheel for cvlib: filename=cvlib-0.2.6-py3-none-any.whl size=10044619 sha256=3128c27c6c144a29eb0a509292f7d50306de53caf3e06dd2621f7075df7be72b
  Stored in directory: /root/.cache/pip/wheels/9b/96/9b/373c07517ffe0bedbebc0813aec1e62cb2bb1dee91b3694895
  Created wheel for progressbar: filename=progressbar-2.5-py3-none-any.whl size=12082 sha256=6e04d8d14463a0b89b3a2d6c33d3f1411de166411a0dd4b96de559516ee01a22
  Stored in directory: /root/.cache/pip/wheels/f0/fd/1f/3e35ed57e94cd8ced38dd46771f1f0f94f65fec548659ed855
Successfully built cvlib progressbar


In [ ]:
!python -m textblob.download_corpora
import nltk
nltk.download('punkt')

In [ ]:
!pip install pyngrok
!ngrok authtoken 22OVDdXxvxRHVNe72i8UcpIniMp_F3nmzJHiTBwZaaeXvSv4

In [ ]:
cd /content/gdrive/MyDrive/Capstone/mmf

/content/gdrive/MyDrive/Capstone/mmf


In [ ]:
!pip install --editable .

Obtaining file:///content/gdrive/MyDrive/Capstone/mmf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Cloning https://github.com/PyTorchLightning/pytorch-lightning (to revision fa0ed17f8) to /tmp/pip-install-sxj23b88/pytorch-lightning_800dc467aaa146008edd373b0594c553
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-install-sxj23b88/pytorch-lightning_800dc467aaa146008edd373b0594c553
  Running command git checkout -q fa0ed17f8
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 869 kB 4.2 MB/s 
     |████████████████████████████████| 64 kB 2.4 MB/s 
     |████████████████████████████████| 831.4 MB 2.5 kB/s 
     |████████████████████████████████| 69 kB 6.3 MB/s 
     |█████████████████████████

In [ ]:
cd /content/gdrive/MyDrive/Capstone/Dashboard/

/content/gdrive/MyDrive/Capstone/Dashboard


Authentication

In [ ]:
# Keys
CONSUMER_KEY = 'x0g63NaebTlGAtpZ2I0Znp6U8'
CONSUMER_SECRET = 'RKTGvBZu7NOu3efWZcnG8zijxNtJoEWzlqS6xLrWyLq0Kv62D2'
OAUTH_TOKEN = '865991910-pqqZNmQINLl5Cju0r3MtFsVhjfJAicz6wQKDuTzg'
OAUTH_TOKEN_SECRET = 'iaqecW3bcPi7BAGVZlTTYABIyqQRt64Fec4sOGv0Oa736'

import tweepy
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN, OAUTH_TOKEN_SECRET)

api = tweepy.API(auth)

import twitter
auth1 = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                           CONSUMER_KEY, CONSUMER_SECRET)

twitter_api = twitter.Twitter(auth=auth1)

Trends, Hashtags, Tweets, User

In [ ]:
import re

WORLD_WOE_ID = 1
US_WOE_ID = 23424977
INDIA_WOE_ID = 23424848

def data_collection(q, count):
    # Import unquote to prevent url encoding errors in next_results
    from urllib.parse import unquote

    search_results = twitter_api.search.tweets(q=q, count=count)

    statuses = search_results['statuses']


    # Iterate through 5 more batches of results by following the cursor
    for _ in range(5):
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
        
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=847960489447628799&q=%23RIPSelena&count=100&include_entities=1
        kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])
    
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
    
    return statuses

def dashboard_data(q, counts, trends):
    
    statuses = data_collection(q, counts)
    
    # India Trends
    india_trends = twitter_api.trends.place(_id=INDIA_WOE_ID)
    count = 0
    for trend in india_trends[0]['trends']:
        trends.append(trend['name'])
        count+=1
        if count >20:
            break
        
    # Tweets 
    status_texts = [ status['text'] 
                 for status in statuses ]
    tweetlist = []
    for i in range(len(status_texts)):
        tweet = str(status_texts[i])
        tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
        tweet = ' '.join(re.sub('RT',' ', tweet).split())
        tweetlist.append(tweet)
    tweet_list = []
    num = 0
    for i in tweetlist:
        tweet_list.append(i)
        num+=1
        if num >20:
            break
  
    # Users
    screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]
    
    mentions = str(screen_names)
    mention = (mentions[:len(mentions)//10])
    
    # Hashtags
    hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]
    
    hashtags = str(hashtags)
    hashtag1 = (hashtags[:len(hashtags)//10])

    return trends, tweet_list, mention, hashtag1

Frequency and Sentiment Data

In [ ]:
import pandas as pd
import csv
header_name = ""

def writing_files(head):
    global header_name
    header_name = head
    with open('./sentiment.csv','w') as file:
        writer = csv.DictWriter(file, fieldnames = header_name)
        writer.writeheader()
    
    with open('./frequency.csv','w') as file:
        writer = csv.DictWriter(file, fieldnames = header_name)
        writer.writeheader()

from tweepy import Stream
from tweepy import StreamListener
from textblob import TextBlob

hashtag_senti = 0
hashtag_freq = 0

import json

class Listener(StreamListener):
    
      def on_data(self, data):
          raw_tweets = json.loads(data)
          tweets = raw_tweets['text']
        
          tweets = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweets).split())
          tweets = ' '.join(re.sub('RT',' ', tweets).split())
        
          blob = TextBlob(tweets.strip())
        
          global hashtag_senti
          global hashtag_freq
                 
          hashtag_sentiment = 0
          hashtag_frequency = 0
          count = 0
          for sent in blob.sentences:
              if hashtag in sent:
                  hashtag_sentiment += sent.sentiment.polarity
                  hashtag_frequency +=  1
                  count += 1
        
          hashtag_senti = hashtag_sentiment/count
          hashtag_freq = hashtag_frequency
        
          with open('./sentiment.csv','a') as file:
              writer = csv.DictWriter(file, fieldnames=header_name)
              #now = datetime.now()
              info = {
                      #'Time': now,
                      hashtag : hashtag_senti
                      }
              writer.writerow(info)
            
          with open('./frequency.csv','a') as file:
              writer = csv.DictWriter(file, fieldnames=header_name)
              #now = datetime.now()
              info = {
                      #'Time': now,
                      hashtag : hashtag_freq
                      }
              writer.writerow(info)
    
      def on_error(self, status):
          print(status)

Image + Text Prediction

In [ ]:
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import numpy as np
import cv2
import cvlib as cv

def face_detect(img_path):

    result = []
    model_path = "/content/gdrive/MyDrive/Capstone/gender_detection.model"

    # read input image
    image = cv2.imread(img_path)

    if image is None:
        print("Could not read input image")
        exit()

    # load pre-trained model
    model = load_model(model_path)

    # detect faces in the image
    face, confidence = cv.detect_face(image)

    classes = ['Male','Female']

    # loop through detected faces
    for idx, f in enumerate(face):

        # get corner points of face rectangle       
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]

        # draw rectangle over face
        cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)

        # crop the detected face region
        face_crop = np.copy(image[startY:endY,startX:endX])

        # preprocessing for gender detection model
        face_crop = cv2.resize(face_crop, (96,96))
        face_crop = face_crop.astype("float") / 255.0
        face_crop = img_to_array(face_crop)
        face_crop = np.expand_dims(face_crop, axis=0)

        # apply gender detection on face
        conf = model.predict(face_crop)[0]

        # get label with max accuracy
        idx = np.argmax(conf)
        label = classes[idx]
        result.append(label)

    return result


In [ ]:
import easyocr
import torch
# Model Saved
MODEL_PATH = './DLmodel.pth'

# Load your trained model
model = torch.load(MODEL_PATH)

def model_predict(img_path):
    
    genders = face_detect(img_path)
    
    reader = easyocr.Reader(['en'], gpu = False)
    result = reader.readtext(img_path,paragraph="False")

    text = ""
    for detection in result: 
        text = text + " " + detection[1]

    output = model.classify(img_path,text)
    hateful = output["label"]
    if hateful:
      if "Female" in genders:
        hateful = 2
      elif "Male" in genders:
        hateful = 3
    return hateful

In [ ]:
# Flask App
from flask import Flask, render_template, request, make_response
from flask_ngrok import run_with_ngrok
app = Flask(__name__)
run_with_ngrok(app)   

# Getting Trends
trends = []

import pickle
text_model = pickle.load(open('MLmodel.pkl','rb'))

# Global Variables
hashtagin = ""

ind=0

jnd=0

    
@app.route("/",methods=["GET", "POST"])
def home():
    global ind,jnd
    ind = 0
    jnd = 0
    senti = pd.read_csv('./sentiment.csv')
    header = senti.columns
    q = header[0] # Query

    count = 100  # Count
    
    ltrends, tweet_list, mention, hashtag1 = dashboard_data(q, count, trends)

    return render_template('index.html', list=ltrends, text=tweet_list, mention=mention, hashtag1=hashtag1)


from time import time
import random

@app.route('/freq1', methods=["GET", "POST"])
def freq1():
    data1 = [time() * 1000, random.randint(0,20)]
    response1 = make_response(json.dumps(data1))
    response1.content_type = 'application/json'
    return response1


@app.route('/senti2', methods=["GET", "POST"])
def senti2():
    data2 = [time() * 1000, random.uniform(-1,1)]
    response2 = make_response(json.dumps(data2))
    response2.content_type = 'application/json'
    return response2


@app.route('/hashtag', methods=['POST'])
def twitter():
    global hashtagin
    hashtagin = request.form['a']

    hashtag = hashtagin
    header_name = [hashtag]
    #headr_name = [hashtag, 'Time']
    
    writing_files(header_name)

    twitter_stream = Stream(auth, Listener())
    twitter_stream.filter(track = [hashtag], is_async=True)
    
    return render_template('live.html')


@app.route('/sentiment', methods=["GET", "POST"])
def sentiment():
    global ind
    sentim = pd.read_csv('./sentiment.csv')
    headr = sentim.columns
    y = sentim[headr[0]].values.tolist()
    data = [time() * 1000, random.uniform(-1,1)]
    ind+=1
    response = make_response(json.dumps(data))
    response.content_type = 'application/json'
    return response


@app.route('/frequency', methods=["GET", "POST"])
def frequency():
    global jnd
    freq = pd.read_csv('./frequency.csv')
    head = freq.columns
    fx = freq[head[0]].values.tolist()
    data = [time() * 1000, random.randint(0,10)]
    jnd+=1
    response = make_response(json.dumps(data))
    response.content_type = 'application/json'
    return response


@app.route("/index")
def index():
    return render_template('index.html')


@app.route("/text_prediction",methods=["GET", "POST"])
def text_prediction():
    a = request.form['a']
    a = str(a)
    x_test = [a]
    text_model = pickle.load(open('MLmodel.pkl','rb'))
    
    val = text_model.predict(x_test)
    val = int(val)
    
    return render_template('text_prediction.html',data=val, text=a)

  
@app.route("/audio")
def audio():
    return render_template('audio.html')

@app.route("/text")
def text():
    return render_template('text.html')

@app.route("/img")
def img():
    return render_template('img.html')

@app.route('/video', methods=['GET'])
def video():
    return render_template('video.html')


import os
from werkzeug.utils import secure_filename
app.config['UPLOAD_FOLDER'] = 'Audio Uploads'
import speech_recognition as sr

@app.route("/uploader" , methods=['GET', 'POST'])
def uploader():
    if request.method=='POST':
        f = request.files['file1']
        f.save(os.path.join(app.config['UPLOAD_FOLDER'], secure_filename(f.filename)))
    
    r = sr.Recognizer() 
    filepath='./Audio Uploads'
    with sr.AudioFile(filepath + '/' + f.filename) as source:
        audio  = r.listen(source)
        text = r.recognize_google(audio)
    
    blo = TextBlob(text.strip())
    pred = []
    for txt in blo.sentences:
        pred.append(txt.sentiment.polarity)
    
    val = int(min(pred))
    if val <= 0:
      ans = 0

    if val > 0:
      ans = 2 

    return render_template('text_prediction.html',data=ans,text=text)


@app.route('/img_predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = "/content/gdrive/MyDrive/Capstone/Dashboard/"
        file_path = os.path.join(
            basepath, 'Image Uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path)
        if preds == 0:
          result = "No Hate Involved"
        elif preds == 1:
          result = "Hateful"
        elif preds == 2:
          result = "Sexist"
        else:
          result = "Racist"
        return result
    return None
  
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import os
import torch
from mamonfight22 import *
from flask import Flask , request , jsonify, redirect, url_for, render_template
from PIL import Image
from io import BytesIO
from time import time

np.random.seed(1234)
# Detect devices
gpu = torch.cuda.is_available()
device = torch.device("cuda" if gpu else "cpu")   # use CPU or GPU
model22 = mamon_videoFightModel(device)
model22.to(device)  


@app.route('/video_predict',methods= ['GET','POST'])
def main_fight(accuracyfight=0.65):
    res_mamon = {}
    if os.path.exists('./Video Uploads/tmp.mp4'):
        os.remove('./Video Uploads/tmp.mp4')
    filev = request.files['file']
    file = open("./Video Uploads/tmp.mp4", "wb")
    file.write(filev.read())
    file.close()
    vid = capture("./Video Uploads/tmp.mp4",40,3,170,170)
    v = np.array([vid])
    v = torch.from_numpy(v)
    v  = v.to(device , dtype=torch.float)
    millis = int(round(time.time() * 1000))
    f , precent = pred_fight(model22,v,acuracy=accuracyfight)
    res_mamon = {'fight':f , 'precentegeoffight':str(precent)}
    millis2 = int(round(time.time() * 1000))
    res_mamon['processing_time'] =  str(millis2-millis)
    resnd = jsonify(res_mamon)
    resnd.status_code = 200
    result = "Hateful" if f else "No Hate Involved"
    return result

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://ad9e-35-221-143-44.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [18/Dec/2021 05:41:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:24] "GET /static/js/scripts.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:24] "GET /static/js/datatables-simple-demo.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:24] "GET /static/assets/demo/chart-bar-demo.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:25] "GET /static/assets/demo/chart-area-demo.js HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:26] "GET /freq1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:26] "GET /senti2 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:26] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [18/Dec/2021 05:41:28] "GET /freq1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:28] "GET /senti2 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:30] "GET /freq1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:31] "GET /senti2 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:33] "GET /freq1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:41:33] "GET /

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
127.0.0.1 - - [18/Dec/2021 05:42:28] "POST /img_predict HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:39] "GET /freq1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:39] "GET /senti2 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:42] "GET /freq1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:42] "GET /senti2 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:44] "GET /freq1 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:44] "GET /senti2 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:42:47] "GET /freq1 HTTP/1.1" 200 -
127.0.


**********************************************************************
  Resource punkt not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt/PY3/english.pickle

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************



Exception in thread Thread-123:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/textblob/decorators.py", line 35, in decorated
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/textblob/tokenizers.py", line 57, in tokenize
    return nltk.tokenize.sent_tokenize(text)
  File "/usr/local/lib/python3.7/dist-packages/nltk/tokenize/__init__.py", line 105, in sent_tokenize
    tokenizer = load('tokenizers/punkt/{0}.pickle'.format(language))
  File "/usr/local/lib/python3.7/dist-packages/nltk/data.py", line 868, in load
    opened_resource = _open(resource_url)
  File "/usr/local/lib/python3.7/dist-packages/nltk/data.py", line 993, in _open
    return find(path_, path + ['']).open()
  File "/usr/local/lib/python3.7/dist-packages/nltk/data.py", line 701, in find
    raise LookupError(resource_not_found)
LookupError: 
**********************************************************************
  Resource punkt not found.
  Please use the


**********************************************************************
  Resource punkt not found.
  Please use the NLTK Downloader to obtain the resource:

  >>> import nltk
  >>> nltk.download('punkt')
  
  For more information see: https://www.nltk.org/data.html

  Attempted to load tokenizers/punkt/PY3/english.pickle

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************



127.0.0.1 - - [18/Dec/2021 05:52:07] "GET /frequency HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:07] "GET /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:09] "GET /frequency HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:10] "GET /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:12] "GET /frequency HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:12] "GET /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:14] "GET /frequency HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:14] "GET /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:17] "GET /frequency HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:17] "GET /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:19] "GET /frequency HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:19] "GET /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:21] "GET /frequency HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:22] "GET /sentiment HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2021 05:52:2

In [ ]:
cd /content/gdrive/MyDrive/Capstone/Dashboard/ssh_files

/content/gdrive/MyDrive/Capstone/Dashboard/ssh_files


In [ ]:
# 2 - Download Ngrok

! wget -q -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip -qq -n ngrok-stable-linux-amd64.zip

# 3 - setup Ngrok - authtoken

#Ask token
print("Get your authtoken from https://dashboard.ngrok.com/auth")
import getpass
authtoken = getpass.getpass()

#Create tunnel
#get_ipython().system_raw('./ngrok authtoken $authtoken && ./ngrok tcp 22 &')
#get_ipython().system_raw('./ngrok authtoken $authtoken && ngrok http http://127.0.0.1:5000/')

Get your authtoken from https://dashboard.ngrok.com/auth
··········


In [ ]:
get_ipython().system_raw('./ngrok authtoken $authtoken && ngrok http http://127.0.0.1:5000/')